In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Please run this code with the kernel reinvent.v3.2

In [4]:
# load dependencies
import sys
import pickle
import os
import shutil
import json
import pandas as pd
import numpy as np
import rdkit.Chem as Chem
from numpy.random import default_rng
import torch
from ast import literal_eval
from torch import nn, optim
from rdkit import Chem
from rdkit.Chem import AllChem
from tdc import Oracle
import subprocess

### First, we need to install the custom reinvent scoring package to support the Bradley-Terry model

In [5]:
# print python version
import sys

# Print Python version
print(f"Python version: {sys.version}")

Python version: 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) 
[GCC 7.5.0]


In [6]:
! pip show reinvent_scoring

Name: reinvent-scoring
Version: 0.0.73
Summary: Scoring functions for Reinvent
Home-page: https://github.com/MolecularAI/reinvent-scoring.git
Author: MolecularAI
Author-email: patronov@gmail.com
License: UNKNOWN
Location: /home/springnuance/reinvent-hitl/reinvent-scoring
Requires: 
Required-by: 


In [7]:
# ! conda install scikit-learn=0.21.3

##### If there already exists reinvent_scoring, we should uninstall it

In [8]:
! pip uninstall -y reinvent_scoring

Found existing installation: reinvent-scoring 0.0.73
Uninstalling reinvent-scoring-0.0.73:
  Successfully uninstalled reinvent-scoring-0.0.73


##### Now we install the custom reinvent scoring package
##### The flag -e means that the package is installed in editable mode, so that changes to the code will be immediately available without reinstalling the package. All package info is stored in the setup.py file.

In [9]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-chemistry"
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-models"

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Running setup.py develop for reinvent-scoring
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-chemistry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-chemistry
    Found existing installation: reinvent-chemistry 0.0.51
    Uninstalling reinvent-chemistry-0.0.51:
      Successfully uninstalled reinvent-chemistry-0.0.51
  Running setup.py develop for reinvent-chemistry
Obtaining file:///home/springnuance/reinvent-hitl/reinvent-models
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-models
    Found existing installation: reinvent-models 0.0.15rc1
    Uninstalling reinvent-models-0.0.15rc1:
      Successfully uninstalled reinvent-models-0.0.15rc1
  Running setup.py develop for reinvent-models


In [10]:
# ! conda install -y scikit-learn=0.21.3
! pip list | grep reinvent_scoring

In [11]:
from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.predict import predict_proba_from_model

def check_create(path):
    """
    Check if the directory exists, if not, create it.
    """
    if not os.path.exists(path):
        os.makedirs(path)
        
def run_HITL_classify(
        seed, reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        base_training_dataset_path, # path to the base training dataset
        model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
        model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
        num_rounds, # number of rounds, corresponding to R in the paper
        num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
        num_queries, # number of molecules shown to the simulated chemist at each iteration
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' 
        sigma_noise, # noise level for simulated chemist's responses
        choose_top_smiles, # number of top scoring molecules to choose for feedback
        training_epochs, # number of epochs for training the model in each HITL iteration
        learning_rate, # learning rate for training the model in each HITL iteration
        ):

    np.random.seed(seed)
    rng = default_rng(seed)
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    for REINVENT_round in range(1, num_rounds + 1):
        check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")
        for HITL_iteration in range(1, num_iters + 1):
            check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}")
    
    # multi-parameter optimization (MPO) loop
    print(f"\nRunning DRD2 (one objective) with rounds {num_rounds}, iters {num_iters}, queries {num_queries}, seed {seed}")
    print(f"Results will be saved at {output_dir}")
    
    base_training_dataset_outputs = load_drd2_dataset(feedback_type=feedback_type, 
                                                 data_path=base_training_dataset_path)

    
    print("Loading initial training datasets successfully")

    # ########################### REINVENT rounds ######################################

    for REINVENT_round in range(1, num_rounds + 1):

        print("=====================================")
        print(f"REINVENT round = {REINVENT_round}")
        
        configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                                REINVENT_round_output_dir=f"{output_dir}/REINVENT_round_{REINVENT_round}", 
                                conf_filename=conf_filename)

        print(f"Creating config file: {configuration_JSON_path}.")

        configuration = json.load(open(f"{configuration_JSON_path}"))

        if REINVENT_round == 1:
            current_model_path = model_pretrained_path
        else:
            # The last iteration's model from the previous round
            current_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"

        configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

        # write the updated configuration file 

        with open(configuration_JSON_path, 'w') as f:
            json.dump(configuration, f, indent=4, sort_keys=True)
    
        print("Run REINVENT")                
        command = f"{reinvent_env}/bin/python"
        script = f"{reinvent_dir}/input.py"
        stderr_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.err"
        stdout_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.out"

        # Construct the full command to run
        cmd = [command, script, configuration_JSON_path]
        # Open the file to which you want to redirect stderr and stdout
        with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
            # Execute the command
            result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
        # Check the result
        print("Exit code:", result.returncode)
        
        #############################################################################
        # REINVENT HAS OUTPUT THE RESULT in path f"{output_dir}/REINVENT_round_{REINVENT_round}/results" #
        #############################################################################
        
        # Get the high scoring molecules
        output_high_score = read_scaffold_result(f"{output_dir}/REINVENT_round_{REINVENT_round}/results/scaffold_memory.csv", 
                                                 choose_top_smiles=choose_top_smiles)
        
        scaffold_df = output_high_score["scaffold_df"]
        smiles = output_high_score["smiles"]
        
        # store molecule indexes selected for feedback
        selected_feedback = np.empty(0).astype(int)

        ########################### HITL_iteration in each REINVENT round #####################
        
        for HITL_iteration in range(1, num_iters + 1): # T number of HITL_iterations

            print("----------------------------------")
            print(f"HITL iteration = {HITL_iteration}")
            
            # Loading feedback model
            if REINVENT_round == 1 and HITL_iteration == 1:
                feedback_model_path = model_pretrained_path
            elif REINVENT_round != 1 and HITL_iteration == 1:
                # The last iteration's model from the previous round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round - 1}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            else:
                # The previous HITL iteration's model from current round
                feedback_model_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration - 1}/{model_pretrained_name}"
            
            feedback_model = load_feedback_model(feedback_type, feedback_model_path)
            
            # print(f"The predicted scores are")
            # predicted_scores = predict_proba_from_model(feedback_type, feedback_model, smiles)
            # print(predicted_scores)

            ######################################################## 
            # Select queries number of smiles with Active Learning #
            ########################################################

            if len(smiles) > num_queries:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, num_queries, list(smiles), 
                                                  selected_feedback, acquisition, rng) 
            else:
                new_queried_smiles_indices = select_query_feedback(feedback_type, feedback_model, 
                                                  scaffold_df, len(smiles), list(smiles), 
                                                  selected_feedback, acquisition, rng)
            
            #print(f"Feedback idx at HITL iteration {HITL_iteration}: {new_queried_smiles_indices}")
            
            new_queried_smiles = [smiles[i] for i in new_queried_smiles_indices]
  
            selected_feedback = np.hstack((selected_feedback, new_queried_smiles_indices))

            new_queried_smiles_human_score = smiles_human_score(new_queried_smiles, sigma_noise)
            
            print(f"Human score at HITL iteration {HITL_iteration}: {new_queried_smiles_human_score}")
            
            # use the augmented training data to retrain the model
            new_queried_fps = np.array([compute_fingerprints(smiles) for smiles in new_queried_smiles])

            iteration_training_dataset_outputs = create_drd2_dataset(feedback_type, 
                                                                      new_queried_smiles, 
                                                                      new_queried_smiles_human_score,
                                                                      new_queried_fps)
            
            if feedback_type == "scoring":
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles'])}")
            else:
                print(f"New queried dataset size: {len(iteration_training_dataset_outputs['smiles_1'])}")
            
            # combining the base training dataset with the new queried dataset
            base_training_dataset_outputs = combine_drd2_dataset(feedback_type, base_training_dataset_outputs, 
                                                                   iteration_training_dataset_outputs)
            
            if feedback_type == "scoring":
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles'])}")
            else:
                print(f"Combined dataset size: {len(base_training_dataset_outputs['smiles_1'])}")
            
            # save augmented training data
            save_drd2_dataset(feedback_type, iteration_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_queried_data.csv")
            save_drd2_dataset(feedback_type, base_training_dataset_outputs, f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            print(f"Saved augmented training data at {output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/iteration_combined_data.csv")
            
            # Retraining the feedback model using the augmented train set
            retrained_feedback_model = retrain_feedback_model(feedback_type, feedback_model, 
                                                              base_training_dataset_outputs,
                                                              training_epochs, learning_rate)
            
            # save the retrained feedback model
            feedback_model_saving_path = f"{output_dir}/REINVENT_round_{REINVENT_round}/HITL_iteration_{HITL_iteration}/{model_pretrained_name}"
            torch.save(retrained_feedback_model.state_dict(), feedback_model_saving_path)
  
            print(f"Saved retrained feedback model at {feedback_model_saving_path}")

def run_REINVENT_round(REINVENT_round,
        reinvent_dir, reinvent_env, output_dir, 
        feedback_type, # scoring, comparing, ranking
        current_model_path, # Path to the pretrained model 
        REINVENT_n_steps, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
        batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
    ):
    
    ################################################
    # DEFINING REINVENT JOBNAME, JOBID, OUTPUT_DIR #
    ################################################

    jobname = "fine-tune predictive component HITL"
    jobid = output_dir
    conf_filename = "config.json"

    # create root output dir
    check_create(output_dir)

    # create HITL round folders to store results
    check_create(f"{output_dir}/REINVENT_round_{REINVENT_round}")

    print("=====================================")
    print(f"REINVENT round = {REINVENT_round}")

    configuration_JSON_path = write_REINVENT_config(feedback_type, reinvent_dir, jobid, jobname, 
                            REINVENT_round_output_dir=f"{output_dir}/REINVENT_round_{REINVENT_round}", 
                            conf_filename=conf_filename)

    print(f"Creating config file: {configuration_JSON_path}.")

    configuration = json.load(open(f"{configuration_JSON_path}"))

    configuration = change_config_json(configuration, REINVENT_n_steps, batch_size, current_model_path)

    # write the updated configuration file 

    with open(configuration_JSON_path, 'w') as f:
        json.dump(configuration, f, indent=4, sort_keys=True)

    print("Run REINVENT")                
    command = f"{reinvent_env}/bin/python"
    script = f"{reinvent_dir}/input.py"
    stderr_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.err"
    stdout_file = f"{output_dir}/REINVENT_round_{REINVENT_round}/run.out"

    # Construct the full command to run
    cmd = [command, script, configuration_JSON_path]
    # Open the file to which you want to redirect stderr and stdout
    with open(stderr_file, 'w') as ferr, open(stdout_file, 'w') as fout:
        # Execute the command
        result = subprocess.run(cmd, text=True, stdout=fout, stderr=ferr)
    # Check the result
    print("Exit code:", result.returncode)

In [12]:
print(os.getcwd())

/home/springnuance/reinvent-hitl/Base-Code-Binh


### Running score regression model

In [17]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

from training_Bradley_Terry_model.bradley_terry import BradleyTerryModel
from training_Rank_ListNet_model.rank_listnet import RankListNetModel
from training_Score_Regression_model.score_regression import ScoreRegressionModel
from scripts.helper import load_drd2_dataset, write_REINVENT_config, change_config_json, \
                    read_scaffold_result, load_feedback_model, smiles_human_score, \
                    compute_fingerprints, retrain_feedback_model,\
                    create_drd2_dataset, combine_drd2_dataset, save_drd2_dataset
                        
from scripts.acquisition import select_query_feedback
from scripts.predict import predict_proba_from_model

seed = 42

# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "scoring" # scoring, comparing, ranking

# feedback type as scoring:
# Given a molecule, what is the probability that the molecule is active regarding DRD2?  

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth"
model_pretrained_name = "score_regression_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 56 # number of molecules, pairs or a set of molecules, depending on the task, 
                 # shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 56 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration


for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        output_dir = f"output_score_regression_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )
            
        if not os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            REINVENT_n_steps_last = 200
            batch_size_last = 64
            # Run the last REINVENT round using the last feedback model
            current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
            run_REINVENT_round(num_rounds+1,
                reinvent_dir, reinvent_env, output_dir, 
                feedback_type, # scoring, comparing, ranking
                current_model_path, # Path to the pretrained model 
                REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
                batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
            )
        else:
            print(f"Already ran the last REINVENT round for {output_dir}")


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

*******************************************************************
Running HITL with random acquisition and noise 0.0
REINVENT round = 4
Creating config file: output_score_regression_n_steps_100/R3_T5_Q56_acq_random_noise_0.0/REINVENT_round_4/config.json.
Run REINVENT
Exit code: 0

*******************************************************************
Running HITL with random acquisition and noise 0.1

Running DRD2 (one objective) with rounds 3, iters 5, queries 56, seed 42
Results will be saved at output_score_regression_n_steps_100/R3_T5_Q56_acq_random_noise_0.1
Loading initial training datasets successfully
REINVENT round = 1
Creating conf

Found local copy...


Loading Score Regression model from /home/springnuance/reinvent-hitl/Base-Code-Binh/training_Score_Regression_model/score_regression_model.pth
Human score at HITL iteration 1: [0.05024990670462845, 0.0, 0.09288112463158087, 0.15896808712441862, 0.0, 0.0, 0.15798413112918813, 0.17365181634270402, 0.0, 0.05457837539282518, 0.0, 0.0, 1.0, 0.0, 0.0, 0.003919934977148619, 0.0, 0.048900175640061265, 0.0, 0.0, 0.14982592788117247, 0.0, 0.008147194014395233, 0.0, 0.0, 0.011221726946224286, 0.0, 0.08077284698120082, 0.7154498457894619, 0.0, 0.0, 0.18693912446189212, 0.039504537253428, 0.06616713886284568, 0.09457012865670847, 0.0, 0.02409572015417394, 0.0, 0.0, 0.020426317392508858, 0.07741258706813461, 0.017533364866355445, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10840945942583731, 0.03597520762216056, 0.0, 0.03273753956916218, 0.0, 0.0, 0.06372363910190118, 0.12827821712409218, 0.0940957131822395]
New queried dataset size: 56
Combined dataset size: 256
Saved augmented training data at output_score_regress

Found local copy...


Epoch 25, Loss: 0.17115193232893944
Saved retrained feedback model at output_score_regression_n_steps_100/R3_T5_Q56_acq_random_noise_0.1/REINVENT_round_1/HITL_iteration_1/score_regression_model.pth
----------------------------------
HITL iteration = 2
Loading Score Regression model from output_score_regression_n_steps_100/R3_T5_Q56_acq_random_noise_0.1/REINVENT_round_1/HITL_iteration_1/score_regression_model.pth
Human score at HITL iteration 2: [0.0, 0.0, 0.04622474268176272, 0.1298293488553393, 0.36871270371863474, 0.0, 0.0, 0.0, 0.09038286018733176, 0.13566679439378826, 0.0, 0.10193442860345905, 0.048890904222364806, 0.0, 0.0417526340859382, 0.15965185080691172, 0.0, 0.15813039202358115, 0.0, 0.08987558456870914, 0.008803746906244313, 0.0, 0.6032013826241795, 0.0, 0.0, 0.03571752294500941, 0.16700161866746713, 0.0, 0.0, 0.0, 0.1598376444625434, 0.052233468552512714, 0.0, 0.051430221160724474, 0.014450960439138141, 0.16888854009485527, 0.0, 0.0, 0.328252291209582, 0.0, 0.0337076601764

...

Batch 1 processed
Human score at HITL iteration 3: [0.01876659909484886, 0.01589127190330459, 0.0, 0.0, 0.0, 0.22826240284524602, 0.14660319781762987, 0.1801908680022996, 0.12520256490720727, 0.0, 0.000583331874906369, 0.10197281462567555, 0.0, 0.12433520152012255, 0.029002818687070234, 0.015553277747923984, 0.1241313553232329, 0.09746168965148772, 0.016938741920975747, 0.16896833883930856, 0.0, 0.11462891919731011, 0.11234548429972212, 0.022080714571484708, 0.08564866072709554, 0.0, 0.14543806042357385, 0.034545144858584835, 0.0007630564509797863, 0.15793628108657268, 0.0, 0.0, 0.0, 0.1136363536374477, 0.0, 0.22851477670132023, 0.0, 0.22268099533188757, 0.07413710522314558, 0.0433640473158445, 0.0, 0.09294896993809082, 0.049271888260398904, 0.1633655467045359, 0.06445812336908771, 0.0680655346490715, 0.01667413728149815, 0.04734079613077699, 0.08381553539208966, 0.0, 0.0, 0.12736176790515524, 0.07012190231567793, 0.03463702486761583, 0.05487875180942305, 0.08779352903905085]
New queri

Found local copy...


Batch 1 processed
Human score at HITL iteration 4: [0.009666839346565856, 0.0, 0.08322733288727016, 0.0, 0.1044680829489492, 0.0, 0.16655837111006638, 0.11090255561880631, 0.08924578080888958, 0.1619119057720534, 0.23019025182375968, 0.16507981387931506, 0.0, 0.0, 0.001160949607934128, 0.06625191239858538, 0.11540870942441432, 0.0, 0.08231948382520984, 0.0, 0.002491027078307187, 0.0, 0.0, 0.0, 0.0, 0.16900698703828598, 0.0, 0.3268810138612411, 0.11297459746559094, 0.08212641974698941, 0.0, 0.13367379531840115, 0.006079024753724613, 0.07584378884203816, 0.3196512612043815, 0.054036817405037026, 0.17857014000606958, 0.0, 0.06014395832780584, 0.240722870941053, 0.0009461015987252686, 0.2448876631771949, 0.13441800092304956, 0.007396129788075245, 0.1268835812830541, 0.16089825234017502, 0.12582380359922166, 0.0, 0.0, 0.03889094382235153, 0.05619946446749738, 0.1257100171325698, 0.0814129074708274, 0.0, 0.10166259247809833, 0.12470138190588433]
New queried dataset size: 56
Combined dataset 

Found local copy...


Batch 1 processed
Human score at HITL iteration 5: [0.11216175280471258, 0.16426078056707077, 0.13957492346649902, 0.10210071593445227, 0.0491661368657439, 0.0, 0.09914452992486719, 0.07695147672791972, 0.08334059173114594, 0.0, 0.0, 0.16149799334489895, 0.052227192625943095, 0.12433277774887104, 0.05901454562465472, 0.05915832196118312, 0.15001108860920837, 0.004578235189871209, 0.06579478570570693, 0.009955179141104352, 0.012733085268377041, 0.02526549566474466, 0.07839608517497988, 0.00830784584526087, 0.18175832056896069, 0.0, 0.03749554359547269, 0.012209602184191107, 0.2008804964469819, 0.07583818566276607, 0.1593671619584721, 0.0, 0.08288773459853449, 0.1451457875739523, 0.06441110693915102, 0.16273074551814373, 0.004453863001571384, 0.1971174520301666, 0.28607252037353365, 0.01989885196764269, 0.011632455871531416, 0.2015211557233787, 0.21413992258467984, 0.0038967052962817703, 0.014164026302022875, 0.02800424712555788, 0.0, 0.0, 0.0, 0.0, 0.05271421927293048, 0.079604181265260

### Running Bradley Terry model

In [18]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42
  
# change these path variables as required
reinvent_dir = "/home/springnuance/reinvent-hitl/Reinvent" # We must use absolute path
reinvent_env = "/home/springnuance/miniconda3/envs/ReinventCommunity" # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "comparing" # scoring, comparing, ranking

# feedback type as comparing:
# Given two molecules, what is the probability that the first molecule is more active than the second molecule regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth"
model_pretrained_name = "bradley_terry_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps
batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 8 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually (num_queries) x (num_queries - 1), since we are comparing each pair of molecules
# As a result, 8 x 7 = 56 pairs of molecules are shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 56 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration

for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        output_dir = f"output_bradley_terry_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )

        if os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            shutil.rmtree(f"{output_dir}/REINVENT_round_{num_rounds+1}")
        
        REINVENT_n_steps_last = 200
        batch_size_last = 64
        # Run the last REINVENT round using the last feedback model
        current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
        run_REINVENT_round(num_rounds+1,
            reinvent_dir, reinvent_env, output_dir, 
            feedback_type, # scoring, comparing, ranking
            current_model_path, # Path to the pretrained model 
            REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
            batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        )


Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

*******************************************************************
Running HITL with random acquisition and noise 0.0

Running DRD2 (one objective) with rounds 3, iters 5, queries 8, seed 42
Results will be saved at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0
Loading initial training datasets successfully
REINVENT round = 1
Creating config file: output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  5006
----------------------------------
HITL iteration = 1


Found local copy...


Loading Bradley Terry model successfully from /home/springnuance/reinvent-hitl/Base-Code-Binh/training_Bradley_Terry_model/bradley_terry_model.pth
Human score at HITL iteration 1: [0.012794059438848513, 0.009591567810738909, 0.00352978573110307, 0.0037539388397401425, 0.002660693177363879, 0.0005912491070823625, 0.006534587595714743, 0.009123164249650971]
New queried dataset size: 56
Combined dataset size: 256
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 5, Loss: 0.1732650939375162
Epoch 10, Loss: 0.17325223796069622
Epoch 15, Loss: 0.1732391994446516
Epoch 20, Loss: 0.17322596907615662


Found local copy...


Epoch 25, Loss: 0.17321249470114708
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
----------------------------------
HITL iteration = 2
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/bradley_terry_model.pth
Human score at HITL iteration 2: [0.007457904482147725, 0.0006749446241312491, 0.001139318899845858, 0.03846690825807318, 0.004862642395150195, 0.0013335642742527075, 0.001574630645522563, 0.00019475991309651273]
New queried dataset size: 56
Combined dataset size: 312
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 5, Loss: 0.2886884758869807
Epoch 10, Loss: 0.28866812338431674
Epoch 15, Loss: 0.28864357868830365
Epoch 20, Loss: 0.2886185919245084


...

B32 processed | Human score at HITL iteration 3: [1.0, 1.0, 0.9966590818356338, 0.6770221207109554, 0.8304379331307014, 0.939233569722167, 0.9354426914011453, 0.639583976550153]
New queried dataset size: 56
Combined dataset size: 928
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/iteration_combined_data.csv
Epoch 5, Loss: 0.6136796902865171
Epoch 10, Loss: 0.6131270807236433
Epoch 15, Loss: 0.6126105058938265
Epoch 20, Loss: 0.6121304053813219
Epoch 25, Loss: 0.6116885356605053
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/bradley_terry_model.pth
----------------------------------
HITL iteration = 4
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_3/bradley_terry_model.pth
B1 processed | B2 processed | B3 processed | B4 processed 

Found local copy...


B32 processed | Human score at HITL iteration 4: [0.7981849992404525, 0.7265224814278957, 0.47080323539691493, 0.9694086645752619, 0.2171303560736953, 0.9925529056798945, 0.7886733028111992, 0.5226299704514035]
New queried dataset size: 56
Combined dataset size: 984
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/iteration_combined_data.csv
Epoch 5, Loss: 0.8724180882175764
Epoch 10, Loss: 0.8717378551761309
Epoch 15, Loss: 0.8711856106917063
Epoch 20, Loss: 0.8706139177083969
Epoch 25, Loss: 0.8700982307394346
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/bradley_terry_model.pth
----------------------------------
HITL iteration = 5
Loading Bradley Terry model successfully from output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_4/bradley_terry_model.pth
B1 processed | B2 process

Found local copy...


B32 processed | Human score at HITL iteration 5: [0.9618755899564847, 1.0, 0.9298568181004594, 0.30974141908886244, 0.2768228821951553, 0.8831190838450398, 1.0, 0.6203974934565578]
New queried dataset size: 56
Combined dataset size: 1040
Saved augmented training data at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/iteration_combined_data.csv
Epoch 5, Loss: 1.3904367610812187
Epoch 10, Loss: 1.3896121457219124
Epoch 15, Loss: 1.388481080532074
Epoch 20, Loss: 1.3876683972775936
Epoch 25, Loss: 1.3874942660331726
Saved retrained feedback model at output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_3/HITL_iteration_5/bradley_terry_model.pth
REINVENT round = 4
Creating config file: output_bradley_terry_n_steps_100/R3_T5_Q8_acq_greedy_noise_0.1/REINVENT_round_4/config.json.
Run REINVENT
Exit code: 0


### Running Rank ListNet model

In [20]:
! pip install -e "/home/springnuance/reinvent-hitl/reinvent-scoring"

seed = 42

# change these path variables as required
reinvent_dir = os.path.expanduser("/home/springnuance/reinvent-hitl/Reinvent") # We must use absolute path
reinvent_env = os.path.expanduser("/home/springnuance/miniconda3/envs/ReinventCommunity") # We must use absolute path

# the performance of the initial model should not be good. Specifically, it should work at 0.5 accuracy 
# If the model is too good, retrain the model to become weaker, we are trying to make the model to learn via HITL

feedback_type = "ranking" # scoring, comparing, ranking

# feedback type as ranking:
# Given N molecules, what are the orders of preference of these molecules regarding DRD2?

base_training_dataset_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/small_drd2_training_data.csv"

model_pretrained_path = "/home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth"
model_pretrained_name = "rank_listnet_model.pth"

num_rounds = 3 # number of rounds, corresponding to R in the paper
num_iters = 5 # number of iterations of showing molecules to the human for feedback at each round
REINVENT_n_steps = 100 # number of REINVENT optimization steps

# Be careful, as the matrix size could get very large, based on binom(num_queries, 3)
# batch_size at 80 could potentially crash the memory

batch_size = 64 # batch size of the reinforcement learning model, or size of scaffold_memory.csv

num_queries = 8 # number of molecules, pairs or a set of molecules, dependig on the task, 
                 # shown to the simulated chemist at each HITL_iteration

# The actual queries is actually binom(num_queries, 3), since we are ranking each set of 3 molecules
# As a result, binom(8, 3) = 56 sets of molecules are shown to the simulated chemist at each HITL_iteration

choose_top_smiles = 56 * num_iters * 2 # number of top molecules to choose from scaffold. 

training_epochs = 25 # number of epochs for training the model in each HITL iteration
learning_rate = 0.000001 # learning rate for training the model in each HITL iteration


for acquisition in ["random", "uncertainty", "greedy"]:
    for sigma_noise in [0.0, 0.1]:
        print("\n*******************************************************************")
        print(f"Running HITL with {acquisition} acquisition and noise {sigma_noise}")
        
        output_dir = f"output_rank_listnet_n_steps_100/R{num_rounds}_T{num_iters}_Q{num_queries}_acq_{acquisition}_noise_{sigma_noise}"

        if os.path.exists(output_dir):
            pass
        else:
            run_HITL_classify(
                    seed, reinvent_dir, reinvent_env, output_dir,
                    feedback_type, # scoring, comparing, ranking
                    base_training_dataset_path, # Path to the pretrained model before REINVENT_round_1/HITL_iteration_1
                    model_pretrained_path, # Path to the pretrained model before REINVENT_round_1
                    model_pretrained_name, # Name of the pretrained model before REINVENT_round_1
                    num_rounds, # number of rounds, corresponding to R in the paper
                    num_iters, # number of iterations of showing molecules to the human for feedback at each round, corresponding to T in the paper
                    num_queries, # number of molecules shown to the simulated chemist at each HITL_iteration
                    REINVENT_n_steps, # number of REINVENT optimization steps
                    batch_size, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
                    acquisition, # acquisition: 'uncertainty', 'random', 'thompson', 'greedy' (if None run with no human interaction)
                    sigma_noise, # noise level for simulated chemist's responses
                    choose_top_smiles, # number of top scoring molecules to choose for feedback
                    training_epochs, # number of epochs for training the model in each HITL iteration
                    learning_rate, # learning rate for training the model in each HITL iteration
            )
        
        if os.path.exists(f"{output_dir}/REINVENT_round_{num_rounds+1}"):
            shutil.rmtree(f"{output_dir}/REINVENT_round_{num_rounds+1}")

        REINVENT_n_steps_last = 200
        batch_size_last = 64
        # Run the last REINVENT round using the last feedback model
        current_model_path = f"{output_dir}/REINVENT_round_{num_rounds}/HITL_iteration_{num_iters}/{model_pretrained_name}"
        run_REINVENT_round(num_rounds+1,
            reinvent_dir, reinvent_env, output_dir, 
            feedback_type, # scoring, comparing, ranking
            current_model_path, # Path to the pretrained model 
            REINVENT_n_steps_last, # number of REINVENT optimization steps. This is not related to the HITL but on the REINVENT side
            batch_size_last, # batch size of the reinforcement learning model, or size of scaffold_memory.csv
        )

Obtaining file:///home/springnuance/reinvent-hitl/reinvent-scoring
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reinvent-scoring
    Found existing installation: reinvent-scoring 0.0.73
    Uninstalling reinvent-scoring-0.0.73:
      Successfully uninstalled reinvent-scoring-0.0.73
  Running setup.py develop for reinvent-scoring

*******************************************************************
Running HITL with random acquisition and noise 0.0

Running DRD2 (one objective) with rounds 3, iters 5, queries 8, seed 42
Results will be saved at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0
Loading initial training datasets successfully
REINVENT round = 1
Creating config file: output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/config.json.
Run REINVENT
Exit code: 0
Number of SMILES in scaffold_memory.csv:  6178
----------------------------------
HITL iteration = 1


Found local copy...


Loading Rank ListNet model successfully from /home/springnuance/reinvent-hitl/Base-Code-Binh/training_Rank_ListNet_model/rank_listnet_model.pth
Human score at HITL iteration 1: [0.0018473915384918111, 0.000545408403508307, 0.06718814304353474, 0.0021518200584256762, 0.00028934689195968526, 8.195340911911775e-05, 0.0025785626866973214, 0.12391968017559284]
New queried dataset size: 56
Combined dataset size: 256
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/iteration_combined_data.csv
Epoch 5, Loss: 0.013398743700236082
Epoch 10, Loss: 0.013380298507399857
Epoch 15, Loss: 0.013361601741053164
Epoch 20, Loss: 0.013342492165975273


Found local copy...


Epoch 25, Loss: 0.013322709477506578
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_model.pth
----------------------------------
HITL iteration = 2
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_1/rank_listnet_model.pth
Human score at HITL iteration 2: [0.00572024758513214, 0.031695467300271174, 0.0017401400849259509, 0.00041346304853797576, 0.0005678800846893891, 0.01702227361599191, 0.0016107511694125705, 0.0006522932250726135]
New queried dataset size: 56
Combined dataset size: 312
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.0/REINVENT_round_1/HITL_iteration_2/iteration_combined_data.csv
Epoch 5, Loss: 0.01800373134513696
Epoch 10, Loss: 0.018220342850933473
Epoch 15, Loss: 0.018264421727508307
Epoch 20, Loss: 0.017944694108640153


...

Found local copy...


Epoch 25, Loss: 0.003234440111555159
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_3/rank_listnet_model.pth
----------------------------------
HITL iteration = 4
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_3/rank_listnet_model.pth
Human score at HITL iteration 4: [0.0, 0.041416005905342446, 0.2949845054321181, 0.9059095922520973, 0.04896098316064684, 0.0, 0.0, 0.004176331103369594]
New queried dataset size: 56
Combined dataset size: 984
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_4/iteration_combined_data.csv
Epoch 5, Loss: 0.005313812813255936
Epoch 10, Loss: 0.0045388269451602055
Epoch 15, Loss: 0.003892940168346589
Epoch 20, Loss: 0.0033443359037240348


Found local copy...


Epoch 25, Loss: 0.0028819089299455905
Saved retrained feedback model at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_4/rank_listnet_model.pth
----------------------------------
HITL iteration = 5
Loading Rank ListNet model successfully from output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_4/rank_listnet_model.pth
Human score at HITL iteration 5: [0.013329089424252014, 0.24723809129950197, 0.0, 0.040029548712795675, 0.0, 0.0, 1.0, 0.08114222383848498]
New queried dataset size: 56
Combined dataset size: 1040
Saved augmented training data at output_rank_listnet_n_steps_100/R3_T5_Q8_acq_random_noise_0.1/REINVENT_round_3/HITL_iteration_5/iteration_combined_data.csv
Epoch 5, Loss: 0.005684805830242112
Epoch 10, Loss: 0.004558196349535137
Epoch 15, Loss: 0.0037710284523200244
Epoch 20, Loss: 0.0031557620386593044
Epoch 25, Loss: 0.0026831096038222313
Saved retrained feedback model at output_rank_list

KeyboardInterrupt: 